In [1]:
# Add project root to Python path
import sys
import os
from pathlib import Path

# Find project root from current notebook location
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # When running from examples folder
    project_root = current_dir.parent.parent
else:
    # When running from project root
    project_root = current_dir

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import required modules
from typing import List, Dict, Any, Optional
from datetime import datetime

from quantbt import (
    TradingStrategy,
    BacktestEngine,  
    
    # Basic modules
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # Order related
    Order, OrderSide, OrderType,
)

In [2]:

class MultiSymbolSMAStrategy(TradingStrategy):
    """
    Hybrid approach:
    - Indicator calculation: Polars vector operations 
    - Signal generation: Dict Native method
    
    Buy: Price above SMA15
    Sell: Price below SMA30  
    """
    
    def __init__(self, buy_sma: int = 15, sell_sma: int = 30, symbols: List[str] = ["KRW-BTC", "KRW-ETH"]):
        super().__init__(
            name="MultiSymbolSMAStrategy",
            config={
                "buy_sma": buy_sma,
                "sell_sma": sell_sma
            },
            position_size_pct=0.8,  # 80% position size
            max_positions=1,
        )
        self.buy_sma = buy_sma
        self.sell_sma = sell_sma
        self.symbols = symbols
        
    def _compute_indicators_for_symbol(self, symbol_data):
        """Calculate moving average indicators per symbol (Polars vector operations)"""
        
        # Ensure time-sorted order
        data = symbol_data.sort("timestamp")
        
        # Calculate simple moving averages
        buy_sma = self.calculate_sma(data["close"], self.buy_sma)
        sell_sma = self.calculate_sma(data["close"], self.sell_sma)
        
        # Add indicator columns
        return data.with_columns([
            buy_sma.alias(f"sma_{self.buy_sma}"),
            sell_sma.alias(f"sma_{self.sell_sma}")
        ])
    
    def generate_signals(self, current_data: Dict[str, Any]) -> List[Order]:
        """Generate signals based on Dict"""
        orders = []
        
        if not self.broker:
            return orders
        
        symbol = current_data['symbol']
        current_price = current_data['close']
        buy_sma = current_data.get(f'sma_{self.buy_sma}')
        sell_sma = current_data.get(f'sma_{self.sell_sma}')
        
        # Skip if indicators are not calculated
        if buy_sma is None or sell_sma is None:
            return orders
        
        current_positions = self.get_current_positions()
        
        # Buy signal: Price above SMA15 + No position
        if current_price > buy_sma and symbol not in current_positions:
            portfolio_value = self.get_portfolio_value()
            quantity = self.calculate_position_size(symbol, current_price, portfolio_value) / len(self.symbols)
            
            if quantity > 0:
                order = Order(
                    symbol=symbol,
                    side=OrderSide.BUY,
                    quantity=quantity,
                    order_type=OrderType.MARKET
                )
                orders.append(order)
                # print(f"📈 Buy signal: {symbol} @ {current_price:,.0f} (SMA{self.buy_sma}: {buy_sma:,.0f})")
        
        # Sell signal: Price below SMA30 + Has position
        elif current_price < sell_sma and symbol in current_positions and current_positions[symbol] > 0:
            order = Order(
                symbol=symbol,
                side=OrderSide.SELL,
                quantity=current_positions[symbol],
                order_type=OrderType.MARKET
            )
            orders.append(order)
            # print(f"📉 Sell signal: {symbol} @ {current_price:,.0f} (SMA{self.sell_sma}: {sell_sma:,.0f})")
        
        return orders

In [3]:


# 1. Upbit data provider
print("🔄 Initializing data provider...")
upbit_provider = UpbitDataProvider()

# 2. Backtesting configuration
config = BacktestConfig(
    symbols=["KRW-BTC", 'KRW-ETH'],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31), 
    timeframe="1d",  # Daily bars (faster than 1-minute bars)
    initial_cash=10_000_000,  # 10 million KRW
    commission_rate=0.0,      # 0% commission (for testing) - use appropriate value for actual backtesting
    slippage_rate=0.0,         # 0% slippage (for testing) - use appropriate value for actual backtesting
    save_portfolio_history=True
)

# 3. SMA strategy
print("⚡ Initializing strategy...")
strategy = MultiSymbolSMAStrategy(
    buy_sma=15,   # Buy: Price above 15-day moving average
    sell_sma=30,   # Sell: Price below 30-day moving average
    symbols=["KRW-BTC", "KRW-ETH"]
)

# 4. Broker configuration
broker = SimpleBroker(
    initial_cash=config.initial_cash,
    commission_rate=config.commission_rate,
    slippage_rate=config.slippage_rate
)

# 5. Dict Native backtest engine (Phase 7)
print("🚀 Initializing Dict Native backtest engine...")
engine = BacktestEngine()  # Using Dict Native engine!
engine.set_strategy(strategy)
engine.set_data_provider(upbit_provider)
engine.set_broker(broker)

🔄 Initializing data provider...
⚡ Initializing strategy...
🚀 Initializing Dict Native backtest engine...


In [4]:
# 7. Results output
result = engine.run(config)
    
# Print result summary
result.print_summary()

처리중... 352/730:  48%|████▊     | 352/730 [00:00<00:00]

처리중... 730/730: 100%|██████████| 730/730 [00:00<00:00]

                 BACKTEST RESULTS SUMMARY
Period          : 2024-01-01 ~ 2024-12-31
Initial Capital : $10,000,000
Final Equity    : $18,106,409
Total Return    : 81.06%
Annual Return   : 81.14%
Volatility      : 28.14%
Sharpe Ratio    : 2.88
Calmar Ratio    : 3.82
Sortino Ratio   : 3.92
Max Drawdown    : 21.25%
Total Trades    : 60
Win Rate        : 50.0%
Profit Factor   : 3.23
Execution Time  : 0.13s


In [5]:
# 1. Portfolio performance chart
result.plot_portfolio_performance()

# 2. Returns distribution 
result.plot_returns_distribution(period="daily")

# 3. Monthly returns heatmap
result.plot_monthly_returns_heatmap()

# 4. Performance comparison table
result.show_performance_comparison()

,Metric,Strategy,Benchmark
0,Total Return (%),81.06,134.35
1,Annual Return (%),81.14,134.49
2,Volatility (%),28.14,46.56
3,Sharpe Ratio,2.88,2.89
4,Calmar Ratio,3.82,4.59
5,Sortino Ratio,3.92,5.06
6,Max Drawdown (%),21.25,29.28
7,Beta,0.43,1.00
8,Alpha,0.24,0.00
9,Total Trades,60,-
